In [1]:
import re
from gurobipy import *
import numpy as np
import itertools

In [2]:
def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()  # Open the file in read mode

    data = {}  # Initialize data as an empty dictionary

    # Each file has number of inputs under which data is stored as a matrix
    numerical_data = re.findall(r'\[([\d\s.]+)\]', content)  # Extract values following each header as numerical values and remove '\t's and '\n's

    sections = ['Orders', 'Allocations', 'DistanceShelfShelf', 'DistancePackagingShelf', 'FullDistanceMatrix']

    for i, section in enumerate(sections):
        data[section] = []
        lines = numerical_data[i].strip().split('\n')
        for line in lines:
            values = line.strip().split()  # Split using spaces
            data[section].append([int(val) for val in values])  # Assuming all values are integers

    return data

# Reading in data and running the entire function

In [3]:
filename = r"Data_Xpress_FullDist.txt"
data = read_file(filename)

# Extracting data into arrays
Orders = data.get('Orders')
Allocations = np.asarray(data.get('Allocations'))
DistanceShelfShelf = np.asarray(data.get('DistanceShelfShelf'))
FullDistanceMatrix = np.asarray(data.get('FullDistanceMatrix'))

NbShelves = 96
Shelves = range(1, NbShelves + 1)

FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 1)
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 0)


# testing for allocation vector where every shelf is filled
allocations_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                    83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                    90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                    21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                    54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                    1, 33, 59, 42, 52,  45, 79, 39, 68, 73, 53]


# testing where only some of the extra shelves are filled
allocations_mid_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                        83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                        90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                        21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                        54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                        1, 33, 59, 42, 52,  45, 79,  0, 68,  0, 53]


In [4]:
def q1_function(allocation_vector):

    def find_closest_product(current_position, products, distances):
        closest_product = None
        min_distance = float('inf')
        for product in products:
            if distances[current_position][product] <= min_distance:
                closest_product = product
                min_distance = distances[current_position][product]
        return closest_product

    def generate_order_lists(order):
        order_lists = []
        tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
        for combination in itertools.product(*[order[i] for i in tuple_indices]):
            new_order = order.copy()
            for i, index in enumerate(tuple_indices):
                new_order[index] = combination[i]
            order_lists.append(new_order)
        return order_lists

    def greedy_order_route(order, distances):
        if not any(isinstance(product, tuple) for product in order):
            visited = [0]
            current_position = 0  
            for k in range(len(order)):
                    closest_product = find_closest_product(current_position, order, distances)
                    visited.append(closest_product)
                    order.remove(closest_product)
                    current_position = closest_product
            visited.append(0)
            OrderDistance = 0
            for i in range(len(visited) - 1):
                OrderDistance += distances[visited[i]][visited[i+1]]
            order_distance_final = OrderDistance
            visited_final = visited

        elif any(isinstance(product, tuple) for product in order):
            order_combinations = generate_order_lists(order)
            order_routes = []
            order_distances = []
            for order in order_combinations:
                visited = [0]
                current_position = 0  
                for k in range(len(order)):
                    closest_product = find_closest_product(current_position, order, distances)
                    visited.append(closest_product)
                    order.remove(closest_product)
                    current_position = closest_product
                visited.append(0)
                order_routes.append(visited)
                OrderDistance = 0
                for i in range(len(visited) - 1):
                    OrderDistance += distances[visited[i]][visited[i+1]]
                order_distances.append(OrderDistance)
            min_idx = order_distances.index(min(order_distances))
            visited_final = order_routes[min_idx]
            order_distance_final = order_distances[min_idx]
                
        return visited_final, order_distance_final
    
    def convert_orders_to_shelf_indices(allocations):
        product_to_shelf = {}
        for shelf_index, product in enumerate(allocations):
            if product != 0:  # Check if the element is not zero
                if product not in product_to_shelf:
                    product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
                else:
                    # If the product already exists in the dictionary, append the new shelf index to the list
                    product_to_shelf[product].append(shelf_index + 1)

        # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
        product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

        OrdersByShelf = []
        for order in Orders:
            order_shelf_indices = []
            for product_index in order:
                if product_index in product_to_shelf_tuples:
                    shelf_indices = product_to_shelf_tuples[product_index]
                    order_shelf_indices.append(shelf_indices)
                else:
                    order_shelf_indices.append(0)  # Product not found in allocation matrix
            OrdersByShelf.append(order_shelf_indices)

        return OrdersByShelf
    

    OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector)
    TotalDistance = 0
    routes = []
    for order in OrdersByShelf:
        visited_order_route, visited_order_dist = greedy_order_route(order, FullDistanceMatrix)
        routes.append(visited_order_route)
        TotalDistance += visited_order_dist
    return 3*TotalDistance, routes

In [5]:
print(q1_function(Allocations[0])[0])
print(q1_function(allocations_full)[0])
print(q1_function(allocations_mid_full)[0])

290706
289200
289728


# Breakdown of the functions individually (commented for checking stuff above)

In [6]:
# def find_closest_product(current_position, products, distances):
#     closest_product = None
#     min_distance = float('inf')
#     for product in products:
#         if distances[current_position][product] <= min_distance:
#             closest_product = product
#             min_distance = distances[current_position][product]
#     return closest_product

# def generate_order_lists(order):
#     order_lists = []
#     tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
#     for combination in itertools.product(*[order[i] for i in tuple_indices]):
#         new_order = order.copy()
#         for i, index in enumerate(tuple_indices):
#             new_order[index] = combination[i]
#         order_lists.append(new_order)
#     return order_lists

# def greedy_order_route(order, distances):
#     if not any(isinstance(product, tuple) for product in order):
#         visited = [0]
#         current_position = 0  
#         for k in range(len(order)):
#                 closest_product = find_closest_product(current_position, order, distances)
#                 visited.append(closest_product)
#                 order.remove(closest_product)
#                 current_position = closest_product
#         visited.append(0)
#         OrderDistance = 0
#         for i in range(len(visited) - 1):
#             OrderDistance += distances[visited[i]][visited[i+1]]
#         order_distance_final = OrderDistance
#         visited_final = visited

#     elif any(isinstance(product, tuple) for product in order):
#         order_combinations = generate_order_lists(order)
#         order_routes = []
#         order_distances = []
#         for order in order_combinations:
#             visited = [0]
#             current_position = 0  
#             for k in range(len(order)):
#                 closest_product = find_closest_product(current_position, order, distances)
#                 visited.append(closest_product)
#                 order.remove(closest_product)
#                 current_position = closest_product
#             visited.append(0)
#             order_routes.append(visited)
#             OrderDistance = 0
#             for i in range(len(visited) - 1):
#                 OrderDistance += distances[visited[i]][visited[i+1]]
#             order_distances.append(OrderDistance)
#         min_idx = order_distances.index(min(order_distances))
#         visited_final = order_routes[min_idx]
#         order_distance_final = order_distances[min_idx]
            
#     return visited_final, order_distance_final

In [7]:
# def convert_orders_to_shelf_indices(allocations):
#     product_to_shelf = {}
#     for shelf_index, product in enumerate(allocations):
#         if product != 0:  # Check if the element is not zero
#             if product not in product_to_shelf:
#                 product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
#             else:
#                 # If the product already exists in the dictionary, append the new shelf index to the list
#                 product_to_shelf[product].append(shelf_index + 1)

#     # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
#     product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

#     OrdersByShelf = []
#     for order in Orders:
#         order_shelf_indices = []
#         for product_index in order:
#             if product_index in product_to_shelf_tuples:
#                 shelf_indices = product_to_shelf_tuples[product_index]
#                 order_shelf_indices.append(shelf_indices)
#             else:
#                 order_shelf_indices.append(0)  # Product not found in allocation matrix
#         OrdersByShelf.append(order_shelf_indices)

#     return OrdersByShelf

In [8]:
# allocations = Allocations[0]

# product_to_shelf = {}
# for i, product_group in enumerate(allocations):
#     if product_group != 0:  # Check if the element is not zero
#         product_to_shelf[product_group] = i

# # Convert order matrix from product indices to shelf indices
# OrdersByShelf1 = []
# for order in Orders:
#     order_shelf_indices = []
#     for product_index in order:
#         if product_index in product_to_shelf:
#             shelf_index = product_to_shelf[product_index]
#             order_shelf_indices.append(shelf_index+1)
#         else:
#             order_shelf_indices.append(0)  # Product not found in allocation matrix
#     OrdersByShelf1.append(order_shelf_indices)

In [9]:
# OrdersByShelf1 = convert_orders_to_shelf_indices(Allocations[0])
# TotalDistance1 = 0
# routes1 = []
# for i, order in enumerate(OrdersByShelf1):
#     visited_order_route1, visited_order_dist1 = greedy_order_route(order, FullDistanceMatrix)
#     routes1.append(visited_order_route1)
    
#     TotalDistance1 += visited_order_dist1
#         # TotalDistance1 += FullDistanceMatrix[visited_order1[i]][visited_order1[i+1]]

# print(TotalDistance1*3)
# routes1

In [10]:
# allocations = test_allocations

# product_to_shelf = {}
# for i, product_group in enumerate(allocations):
#     if product_group != 0:  # Check if the element is not zero
#         product_to_shelf[product_group] = i

# # Convert order matrix from product indices to shelf indices
# OrdersByShelf2 = []
# for order in Orders:
#     order_shelf_indices = []
#     for product_index in order:
#         if product_index in product_to_shelf:
#             shelf_index = product_to_shelf[product_index]
#             order_shelf_indices.append(shelf_index+1)
#         else:
#             order_shelf_indices.append(0)  # Product not found in allocation matrix
#     OrdersByShelf2.append(order_shelf_indices)

In [11]:
# allocation_matrix = [0] * 96  # Initialize with zeros
# product_to_shelf = {}

# for shelf_index, product in enumerate(test_allocations):
#     if product not in product_to_shelf:
#         product_to_shelf[product] = [shelf_index+1]  # Initialize with a list containing the current shelf index
#     else:
#         # If the product already exists in the dictionary, append the new shelf index to the list
#         product_to_shelf[product].append(shelf_index + 1)

# # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
# product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

# OrdersByShelf2 = []
# for order in Orders:
#     order_shelf_indices = []
#     for product_index in order:
#         if product_index in product_to_shelf_tuples:
#             shelf_indices = product_to_shelf_tuples[product_index]
#             order_shelf_indices.append(shelf_indices)
#         else:
#             order_shelf_indices.append(0)  # Product not found in allocation matrix
#     OrdersByShelf2.append(order_shelf_indices)

In [12]:
# OrdersByShelf2 = convert_orders_to_shelf_indices(allocations_full)
# TotalDistance2 = 0
# routes2 = []
# for i, order in enumerate(OrdersByShelf2):
#     visited_order_route2, visited_order_dist2 = greedy_order_route(order, FullDistanceMatrix)
#     routes2.append(visited_order_route2)
    
#     TotalDistance2 += visited_order_dist2
#         # TotalDistance1 += FullDistanceMatrix[visited_order1[i]][visited_order1[i+1]]

# print(TotalDistance2*3)
# routes2

In [13]:
# OrdersByShelf3 = convert_orders_to_shelf_indices(allocations_mid_full)
# TotalDistance3 = 0
# routes3 = []
# for i, order in enumerate(OrdersByShelf3):
#     visited_order_route3, visited_order_dist3 = greedy_order_route(order, FullDistanceMatrix)
#     routes3.append(visited_order_route3)
    
#     TotalDistance3 += visited_order_dist3

# print(TotalDistance3*3)
# routes3

In [14]:
# maxes = []
# for i in routes2:
#     maxes.append(max(i))
# max(maxes)

In [15]:
# print(f"Total distance: {3*TotalDistance1}")
# print(f"Total distance: {3*TotalDistance2}")
# print(f"Total distance: {3*TotalDistance3}")